# 00 Pipeline Demo


このノートブックは `config/config.example.yaml` を読み込み、synthetic データで手法を単体検証します。

In [1]:
import sys
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

PROJECT_ROOT = Path.cwd()
if not (PROJECT_ROOT / "config" / "config.example.yaml").exists():
    PROJECT_ROOT = PROJECT_ROOT.parent
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

from src.config import load_config
from src.utils.calendar import month_end_dates, quarter_end_dates
from src.utils.io import save_parquet, save_csv, save_figure
from src.utils.validation import assert_weights_sum_to_one, assert_no_lookahead

cfg = load_config(PROJECT_ROOT / "config" / "config.example.yaml")
np.random.seed(cfg["project"]["seed"])

for rel in ["outputs/logs", "outputs/signals", "outputs/weights", "outputs/diagnostics", "outputs/figures"]:
    (PROJECT_ROOT / rel).mkdir(parents=True, exist_ok=True)

def make_synthetic(cfg):
    dates = pd.bdate_range(cfg["data"]["date_start"], cfg["data"]["date_end"])
    themes = cfg["data"]["assets"]["theme_ids"]
    n, m = len(dates), len(themes)

    factors = np.random.normal(0.0, 0.004, size=(n, 3))
    loadings = np.random.uniform(-1.0, 1.0, size=(m, 3))
    eps = np.random.normal(0.0, 0.01, size=(n, m))
    returns = pd.DataFrame(factors @ loadings.T + eps, index=dates, columns=themes)
    prices = 100.0 * (1.0 + returns).cumprod()

    flow_common = np.random.normal(0.0, 1.0, size=n)
    flow_noise = np.random.normal(0.0, 0.6, size=(n, m))
    flow = pd.DataFrame(flow_common[:, None] + flow_noise, index=dates, columns=themes)

    theme_scores = flow.rolling(21, min_periods=5).mean() + np.random.normal(0.0, 0.2, size=(n, m))
    stale_cols = themes[2::2]
    if len(stale_cols) > 0:
        theme_scores.loc[dates[-120:], stale_cols] = np.nan

    if cfg["calendar"]["rebalance"] == "Q":
        rebalance_dates = quarter_end_dates(dates)
    else:
        rebalance_dates = month_end_dates(dates)

    return dates, themes, returns, prices, flow, theme_scores, rebalance_dates


In [2]:
from src.signals.theme_selection_msci import compute_theme_rank
from src.signals.trend_scanning import trend_scanning
from src.signals.flow_diagnostics import compute_flow_pressure_score, local_projection_irf
from src.signals.flow_sorted_factor import build_flow_sorted_portfolios
from src.signals.unexpected_flow import decompose_flow, predict_returns_with_unexpected_flow
from src.risk.crowding import compute_crowding_components, aggregate_crowding, apply_crowding_overlay
from src.backtest.engine import backtest_daily, compute_metrics

dates, themes, returns, prices, flow, theme_scores, rebalance_dates = make_synthetic(cfg)

selected_themes, theme_weights_raw, debug_df = compute_theme_rank(
    theme_scores=theme_scores,
    theme_index_returns=returns,
    rebalance_dates=rebalance_dates,
    top_k=cfg["msci_theme_select"]["top_k"],
    stale_months=cfg["msci_theme_select"]["stale_months"],
    window_days=cfg["msci_theme_select"]["fallback"]["window_days"],
)

_ = trend_scanning(
    prices=prices[themes[0]],
    lookback_min=cfg["trend_scanning"]["lookback_min"],
    lookback_max=cfg["trend_scanning"]["lookback_max"],
    step=cfg["trend_scanning"]["step"],
    use_log=cfg["trend_scanning"]["use_log"],
)

pressure = pd.DataFrame({
    c: compute_flow_pressure_score(flow[c], window=cfg["flow_diagnostics"]["flow_window_days"], z=True) for c in themes
}, index=dates)
_ = local_projection_irf(
    returns=returns[themes[0]],
    shock=pressure.mean(axis=1),
    horizons=range(1, cfg["flow_diagnostics"]["irf_horizons"] + 1),
    nw_lags=cfg["flow_diagnostics"]["nw_lags"],
)

_ = build_flow_sorted_portfolios(
    returns=returns,
    flow=flow,
    rebalance_dates=rebalance_dates,
    q=cfg["flow_sorted_factor"]["q"],
    weighting=cfg["flow_sorted_factor"]["weighting"],
    lag=cfg["flow_sorted_factor"]["lag_days"],
)

features_dict = {
    "ret_21": returns.rolling(21, min_periods=5).sum(),
    "vol_21": returns.rolling(21, min_periods=5).std(ddof=0),
    "mom_5": returns.rolling(5, min_periods=3).sum(),
}
_, unexpected_flow, _ = decompose_flow(
    flow=flow,
    features_dict=features_dict,
    model=cfg["unexpected_flow"]["model"],
    standardize=cfg["unexpected_flow"]["standardize"],
)
_ = predict_returns_with_unexpected_flow(
    returns=returns,
    unexpected_flow=unexpected_flow,
    horizon=cfg["unexpected_flow"]["horizon_days"],
)

components = compute_crowding_components(
    prices=prices,
    calendar=cfg["calendar"],
    window=cfg["crowding"]["window_days"],
)
score = aggregate_crowding(components, method="mean", handle_missing="renormalize")
theme_weights_overlay = apply_crowding_overlay(
    base_weights=theme_weights_raw,
    crowding_score=score,
    cap_quantile=cfg["crowding"]["overlay"]["cap_quantile"],
    scale=cfg["crowding"]["overlay"]["scale"],
)

active = theme_weights_overlay.loc[theme_weights_overlay.sum(axis=1) > 0]
assert_weights_sum_to_one(active)
assert_no_lookahead(theme_weights_overlay, returns, lag=cfg["calendar"]["trading_lag_days"])

portfolio_ret = backtest_daily(
    returns=returns,
    weights=theme_weights_overlay,
    cost_bps=cfg["backtest"]["cost_bps"],
    lag=cfg["calendar"]["trading_lag_days"],
)
metrics = compute_metrics(portfolio_ret)

save_parquet(theme_weights_raw, PROJECT_ROOT / "outputs/weights/00_theme_weights_raw.parquet")
save_parquet(theme_weights_overlay, PROJECT_ROOT / "outputs/weights/00_theme_weights_overlay.parquet")
save_parquet(theme_weights_overlay.shift(cfg["calendar"]["trading_lag_days"]).fillna(0.0), PROJECT_ROOT / "outputs/weights/00_portfolio_weights_applied.parquet")
save_csv(metrics.to_frame(name="value"), PROJECT_ROOT / "outputs/diagnostics/00_metrics.csv")

fig, ax = plt.subplots(figsize=(10, 4))
(1.0 + portfolio_ret.fillna(0.0)).cumprod().plot(ax=ax)
ax.set_title("00 Pipeline Equity Curve")
ax.set_ylabel("Equity")
save_figure(fig, PROJECT_ROOT / "outputs/figures/00_equity_curve.png")
plt.close(fig)

print(metrics)


CAGR     -0.023499
Vol       0.084340
Sharpe   -0.239756
MaxDD    -0.442143
dtype: float64


In [3]:
required = [
    "outputs/weights/00_theme_weights_raw.parquet",
    "outputs/weights/00_theme_weights_overlay.parquet",
    "outputs/weights/00_portfolio_weights_applied.parquet",
    "outputs/diagnostics/00_metrics.csv",
    "outputs/figures/00_equity_curve.png",
]
status = "OK" if all((PROJECT_ROOT / p).exists() for p in required) else "NG"
checks = ", ".join([f"{p}:{(PROJECT_ROOT / p).exists()}" for p in required])
line = f"Notebook=00_pipeline_demo.ipynb, {status}, 生成ファイル確認={checks}\n"
with (PROJECT_ROOT / "outputs/logs/qa_checklist.txt").open("a", encoding="utf-8") as f:
    f.write(line)
print(line)


Notebook=00_pipeline_demo.ipynb, OK, 生成ファイル確認=outputs/weights/00_theme_weights_raw.parquet:True, outputs/weights/00_theme_weights_overlay.parquet:True, outputs/weights/00_portfolio_weights_applied.parquet:True, outputs/diagnostics/00_metrics.csv:True, outputs/figures/00_equity_curve.png:True

